<a href="https://colab.research.google.com/github/Adria100/clin_IQ/blob/main/4_Evaluation_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.metrics import f1_score
import pandas as pd

#K approach

In [1]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=152e6d16ad7cdf3e6d34c45d6e0207ae2402f27d9743c6e509897b12ce8cd2d4
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [ ]:
!pip install spacy transformers scikit-learn rouge-score nltk textacy
!python -m spacy download en_core_web_md

In [8]:
import json
import spacy
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
from nltk.tokenize import sent_tokenize
from transformers import pipeline

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# Update file path


In [ ]:
nlp = spacy.load('en_core_web_md')

# Hugging Face pipeline for entailment (textual consistency)
entailment_pipeline = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

For multi-hop evaluation we have looked at evaluation approaches used in literature:

-BERTScore, BLEU, METEOR  and Levenshtein Ratio
Lemesle, Q., Chevelu, J., Martin, P., Lolive, D., Delhay, A., & Barbot, N. (2025, January). Paraphrase Generation Evaluation Powered by an LLM: A Semantic Metric, Not a Lexical One. In Proceedings of the 31st International Conference on Computational Linguistics (pp. 8057-8087).

-Cosine Similarity
Patwardhan, A., Vaidya, V., & Kundu, A. (2024, October). Automated Consistency Analysis of LLMs. In 2024 IEEE 6th International Conference on Trust, Privacy and Security in Intelligent Systems, and Applications (TPS-ISA) (pp. 118-127). IEEE.

-G-Eval
Liu, Y., Iter, D., Xu, Y., Wang, S., Xu, R., & Zhu, C. (2023). G-eval: NLG evaluation using gpt-4 with better human alignment. arXiv preprint arXiv:2303.16634.

-ROUGE
Briman, M. K. H., & Yildiz, B. (2024). Beyond ROUGE: A comprehensive evaluation metric for abstractive summarization leveraging similarity, entailment, and acceptability. International Journal on Artificial Intelligence Tools. DOI: https://doi. org/10.1142 S.


In [ ]:
def evaluate_classification(y_true, y_pred):
    """
    Calculate accuracy, precision, and recall for letter classification.

    Parameters:
        y_true (list): True labels
        y_pred (list): Predicted labels

    Returns:
        dict: A dictionary with accuracy, precision, and recall
    """
    return {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision': precision_score(y_true, y_pred, average='macro', zero_division=0),
        'recall': recall_score(y_true, y_pred, average='macro', zero_division=0)
    }
#BERTScore

#METEOR

#Levenshtein

#Cosine Similarity
def compute_cosine_similarity(reference, prediction):
        """
        Compute cosine similarity between two texts using their embeddings.
        """
        doc1 = nlp(text1)
        doc2 = nlp(text2)
        return cosine_similarity([doc1.vector], [doc2.vector])[0][0]

#G-Eval
def evaluate_reasoning_flow(reference, prediction):
        """
        Evaluate coherence and reasoning flow (G-Eval metric).
        This uses cosine similarity between consecutive sentences.
        """
        sentences = sent_tokenize(text)
        flow_scores = []

        for i in range(1, len(sentences)):
            score = self.compute_cosine_similarity(sentences[i-1], sentences[i])
            flow_scores.append(score)

        # Return average reasoning flow score
        return np.mean(flow_scores) if flow_scores else 0

#BLEU
def compute_bleu(reference, prediction):
    """
    Compute BLEU score between a reference and a prediction.

    Parameters:
        reference (str): The ground truth answer.
        prediction (str): The model's predicted answer.

    Returns:
        float: BLEU score (0 to 1)
    """
    # Tokenize by splitting on whitespace
    reference_tokens = [reference.split()]
    prediction_tokens = prediction.split()

    # Smoothing helps avoid zero scores for short predictions
    smoothie = SmoothingFunction().method4

    return sentence_bleu(reference_tokens, prediction_tokens, smoothing_function=smoothie)

#ROUGE
def compute_rouge(reference, prediction):
    """
    Compute ROUGE scores between a reference and a prediction.

    Parameters:
        reference (str): The ground truth answer.
        prediction (str): The model's predicted answer.

    Returns:
        dict: ROUGE-1, ROUGE-2, and ROUGE-L F1 scores
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, prediction)

    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }


class EvaluationSuite():
    """
    Class that contains the evaluation scripts for each question type.
    """
    def evaluate_MC(self, predictions, ground_truth):
        """
        Evaluates the accuracy of multiple-choice predictions.

        This function compares a list of predicted answers (e.g., "A", "B", "C", etc.)
        against the corresponding ground-truth labels and computes overall accuracy.

        Args:
            predictions (List[str]): A list of predicted choices (e.g., ["B", "C", "A", ...]).
            ground_truth (List[str]): A list of correct choices (same format and length as predictions).

        Returns:
            Saves a confusion matrix
            dict: A dictionary containing:
            - 'accuracy' (float): Overall classification accuracy.
            - 'precision' (float): Macro-averaged precision across all classes.
            - 'recall' (float): Macro-averaged recall across all classes.
        """

        # Generate confusion matrix
        labels = sorted(list(set(ground_truth + predictions)))  # Get all possible classes
        cm = confusion_matrix(ground_truth, predictions, labels=labels)

        # Display the matrix
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(cmap='Blues', xticks_rotation=45)
        plt.title("Confusion Matrix")
        plt.tight_layout()
        plt.savefig("output/confusion_matrix_MC.png", dpi=300)  # You can change filename and resolution
        plt.show()

        return evaluate_classification(ground_truth, predictions)


    def evaluate_FT(self, predictions, ground_truth):

        labels = sorted(list(set(ground_truth + predictions)))

        # Compute confusion matrix
        cm = confusion_matrix(ground_truth, predictions, labels=labels)

        # Display the confusion matrix using ConfusionMatrixDisplay
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
        disp.plot(cmap='Blues', xticks_rotation=45)
        plt.title("Confusion Matrix")
        plt.tight_layout()

        # Save the confusion matrix plot
        plt.savefig("output/confusion_matrix_FT.png", dpi=300)  # You can change filename and resolution
        plt.show()  # Show the plot

        # Compute and return classification metrics
        return self.evaluate_classification(ground_truth, predictions)

    def evaluate_SA(self, predictions, ground_truth):
      # Ensure that predictions and ground_truth have the same length
        assert len(predictions) == len(ground_truth), "Predictions and ground_truth must have the same length."

        labels = sorted(list(set(ground_truth + predictions)))
        BLEU_scores = []
        ROUGE_scores = []

        # Loop through predictions and calculate BLEU and ROUGE scores
        for idx, prediction in enumerate(predictions):
            if prediction == 'N/A':
                continue  # Skip 'N/A' predictions

            # Print the ground truth and predicted answers for debugging
            print("Ground truth: ", ground_truth[idx])
            print("Prediction: ", prediction)

            # Compute BLEU and ROUGE scores
            bleu_score = compute_bleu(ground_truth[idx], prediction)
            rouge_score = compute_rouge(ground_truth[idx], prediction)

            # Print the scores
            print("BLEU: ", bleu_score)
            print("ROUGE: ", rouge_score)

            # Append scores to the lists
            BLEU_scores.append(bleu_score)
            ROUGE_scores.append(rouge_score)
            avg_bleu = sum(BLEU_scores) / len(BLEU_scores) if BLEU_scores else 0
            avg_rouge = sum(ROUGE_scores) / len(ROUGE_scores) if ROUGE_scores else 0

            return avg_rouge, avg_bleu
    def evaluate_MH(self,predictions, ground_truth):
        # Ensure that predictions and ground_truth have the same length
        assert len(predictions) == len(ground_truth), "Predictions and ground_truth must have the same length."

        labels = sorted(list(set(ground_truth + predictions)))
        BLEU_scores = []
        ROUGE_scores = []
        similarity_scores = []
        coherence_scores = []

        # Loop through predictions and calculate BLEU and ROUGE scores
        for idx, prediction in enumerate(predictions):
            if prediction == 'N/A':
                continue  # Skip 'N/A' predictions

            # Print the ground truth and predicted answers for debugging
            print("Ground truth: ", ground_truth[idx])
            print("Prediction: ", prediction)

            # Compute BLEU and ROUGE scores
            bleu_score = compute_bleu(ground_truth[idx], prediction)
            rouge_score = compute_rouge(ground_truth[idx], prediction)
            similarity = compute_cosine_similarity(ground_truth[idx], prediction)
            coherence = evaluate_reasoning_flow(prediction)

            # Print the scores
            print("BLEU: ", bleu_score)
            print("ROUGE: ", rouge_score)
            print(f"Average Cosine Similarity: similarity)
            print(f"Average Coherence: {evaluation_results['Coherence Score']}")

            # Append scores to the lists
            BLEU_scores.append(bleu_score)
            ROUGE_scores.append(rouge_score)
            avg_bleu = sum(BLEU_scores) / len(BLEU_scores) if BLEU_scores else 0
            avg_rouge = sum(ROUGE_scores) / len(ROUGE_scores) if ROUGE_scores else 0

            return avg_rouge, avg_bleu